In [ ]:
import dolfinx as dfx

import gmsh
import pygmsh

import meshio

%matplotlib widget
import matplotlib.pyplot as plt

from mpi4py import MPI

import numpy as np

In [ ]:
mesh_comm = MPI.COMM_WORLD

model_rank = 0

L = 1.

resolution = 0.01

# Initialize empty geometry using the build in kernel in GMSH
geometry = pygmsh.geo.Geometry()
# Fetch model we would like to add data to
model = geometry.__enter__()

# Add points with finer resolution on left side
points = [model.add_point((0, 0, 0), mesh_size=resolution),
          model.add_point((L, 0, 0), mesh_size=0.1 * resolution)]

line = model.add_line(points[0], points[1])

# Call gmsh kernel before add physical entities
model.synchronize()

model.add_physical([line], "Volume")

geometry.generate_mesh(dim=2)
gmsh.write("mesh.msh")
gmsh.clear()
geometry.__exit__()


In [ ]:
mesh_from_file = meshio.read("mesh.msh")


In [ ]:
def create_mesh(mesh, cell_type, prune_z=False):

    cells = mesh.get_cells_type(cell_type)
    cell_data = mesh.get_cell_data("gmsh:physical", cell_type)

    points = mesh.points[:, :2] if prune_z else mesh.points

    out_mesh = meshio.Mesh(points=points, cells={cell_type: cells}, cell_data={
                           "name_to_read": [cell_data]})
    return out_mesh


In [ ]:
line_mesh = create_mesh(mesh_from_file, "line", prune_z=True)
meshio.write("line_mesh.xdmf", line_mesh)

In [ ]:
with dfx.io.XDMFFile(mesh_comm, "line_mesh.xdmf", 'r') as file:
    mesh = file.read_mesh(name="Grid")

In [ ]:
V = dfx.fem.FunctionSpace(mesh, ("CG", 1))

cells, types, x = dfx.plot.vtk_mesh(V)

In [ ]:
plt.plot(x[:, 0], 'x')